In [1]:
import os
from nltk import word_tokenize
from nltk.tokenize import WordPunctTokenizer
import string
import re
from nltk.corpus import stopwords
from pattern.en import spelling

In [2]:
def get_files(path):
    return os.listdir(path)

In [7]:
path_list = ['review_polarity/txt_sentoken/pos/', 'review_polarity/txt_sentoken/neg/']


In [8]:
def clean(word):
    return word.translate(re.sub(r'\d+', '', word).maketrans('', '', string.punctuation)).lower()

In [14]:
def clear_document(sentences):
    new_words = ''
    for sentence in sentences:
        new_words += sentence.strip()
    new_words = clean(new_words)
    tokinezed = word_tokenize(new_words)
    # correct_sentence = [spelling.suggest(i)[0][0] for i in tokinezed]
    stops = stopwords.words('english')
    filtered_words = [word for word in tokinezed if word not in stops]
    return filtered_words

In [15]:
pos = []
neg = []
for path in path_list:
    files = get_files(path)
    for file in files:
        with open(path+file) as f:
            words = f.readlines()
        if path[-4:-1] == 'pos':
            pos += words
        else:
            neg += words

In [16]:
cleaned_document = clear_document(pos)

In [17]:
all_documents = []
for path in path_list:
    files = get_files(path)
    for file in files:
        with open(path+file) as f:
            words = f.readlines()
        cleaned_document = clear_document(words)
        all_documents.append((clear_document, path[-4:-1]))

